In [16]:
!poetry add mysql-connector-python

Using version ^8.1.0 for mysql-connector-python

Updating dependencies
Resolving dependencies... (1.1s)

No dependencies to install or update

Writing lock file


In [26]:
# Import the MySQL Connector/Python Pool
from mysql.connector.pooling import MySQLConnectionPool
from mysql.connector import Error

dbconfig = {
    "database":"little_lemon",
    "user" : "megaalpha",
    "password" : ""
}

try:
    pool = MySQLConnectionPool(pool_name = "pool_a",
                           pool_size = 2, #default is 5
                           **dbconfig)
    print("The connection pool is created with a name: ",pool.pool_name)
    print("The pool size is:",pool.pool_size)

except Error as er:
    print("Error code:", er.errno)
    print("Error message:", er.msg)
    sys.exit(3)

# Create cursor object to communicate with entire MySQL database
try:
        connection = pool.get_connection()
except:
        # To add connection to the pool, the connection must be of MySQLConnection instance 
        # Also possible to create via connect module and need the import below
        import mysql.connector as connector
        print("No more connections are available.")
        print("Adding new connection in the pool.")
        
        # Create a connection
        connection=connector.connect(**dbconfig)
        # Add the connection into the pool
        pool.add_connection(cnx=connection)
        print("A new connection is added in the pool.\n")
        
        new_connection = pool.get_connection()
        connection = new_connection

cursor = connection.cursor()
print("Cursor is created to communicate with the MySQL using Python.\n")

The connection pool is created with a name:  pool_a
The pool size is: 2
Cursor is created to communicate with the MySQL using Python.



In [22]:
# Create database and checking all that we have!
cursor.execute("CREATE DATABASE IF NOT EXISTS little_lemon")
print("The database little_lemon is created.\n")
# Set little_lemon database for use
cursor.execute("USE little_lemon")

# Confirm the database in use
assert connection.database == "little_lemon"

The database little_lemon is created.



In [27]:
#MenuItems table
create_menuitem_table = """CREATE TABLE IF NOT EXISTS MenuItems (
ItemID INT AUTO_INCREMENT,
Name VARCHAR(200),
Type VARCHAR(100),
Price INT,
PRIMARY KEY (ItemID)
);"""

create_menu_table = """CREATE TABLE IF NOT EXISTS Menus (
MenuID INT,
ItemID INT,
Cuisine VARCHAR(100),
PRIMARY KEY (MenuID,ItemID)
);"""

create_booking_table = """CREATE TABLE IF NOT EXISTS Bookings (
BookingID INT AUTO_INCREMENT,
TableNo INT,
GuestFirstName VARCHAR(100) NOT NULL,
GuestLastName VARCHAR(100) NOT NULL,
BookingSlot TIME NOT NULL,
EmployeeID INT,
PRIMARY KEY (BookingID)
);"""

create_orders_table = """CREATE TABLE IF NOT EXISTS Orders (
OrderID INT,
TableNo INT,
MenuID INT,
BookingID INT,
BillAmount INT,
Quantity INT,
PRIMARY KEY (OrderID,TableNo)
);"""

create_employees_table = """CREATE TABLE IF NOT EXISTS Employees (
EmployeeID INT AUTO_INCREMENT PRIMARY KEY,
Name VARCHAR(100),
Role VARCHAR(50),
Address VARCHAR(70),
Contact_Number INT,
Email VARCHAR(50),
Annual_Salary VARCHAR(20)
);"""

# Create MenuItems table
cursor.execute(create_menuitem_table)
print("Creating menu item table")

# Create Menu table
cursor.execute(create_menu_table)
print("Creating menu table")

# Create Bookings table
cursor.execute(create_booking_table)
print("Creating booking table")

# Create Orders table
cursor.execute(create_orders_table)
print("Creating orders table")

# Create Employees table
cursor.execute(create_employees_table)
print("Creating employees table")

Creating menu item table
Creating menu table
Creating booking table
Creating orders table
Creating employees table


In [28]:
#*******************************************************#
# Insert query to populate "MenuItems" table:
#*******************************************************#
insert_menuitems="""
INSERT IGNORE INTO MenuItems (ItemID, Name, Type, Price)
VALUES
(1, 'Olives','Starters',5),
(2, 'Flatbread','Starters', 5),
(3, 'Minestrone', 'Starters', 8),
(4, 'Tomato bread','Starters', 8),
(5, 'Falafel', 'Starters', 7),
(6, 'Hummus', 'Starters', 5),
(7, 'Greek salad', 'Main Courses', 15),
(8, 'Bean soup', 'Main Courses', 12),
(9, 'Pizza', 'Main Courses', 15),
(10, 'Greek yoghurt','Desserts', 7),
(11, 'Ice cream', 'Desserts', 6),
(12, 'Cheesecake', 'Desserts', 4),
(13, 'Athens White wine', 'Drinks', 25),
(14, 'Corfu Red Wine', 'Drinks', 30),
(15, 'Turkish Coffee', 'Drinks', 10),
(16, 'Turkish Coffee', 'Drinks', 10),
(17, 'Kabasa', 'Main Courses', 17);"""

#*******************************************************#
# Insert query to populate "Menu" table:
#*******************************************************#
insert_menu="""
INSERT IGNORE INTO Menus (MenuID,ItemID,Cuisine)
VALUES
(1, 1, 'Greek'),
(1, 7, 'Greek'),
(1, 10, 'Greek'),
(1, 13, 'Greek'),
(2, 3, 'Italian'),
(2, 9, 'Italian'),
(2, 12, 'Italian'),
(2, 15, 'Italian'),
(3, 5, 'Turkish'),
(3, 17, 'Turkish'),
(3, 11, 'Turkish'),
(3, 16, 'Turkish');"""

#*******************************************************#
# Insert query to populate "Bookings" table:
#*******************************************************#
insert_bookings="""
INSERT IGNORE INTO Bookings (BookingID, TableNo, GuestFirstName, 
GuestLastName, BookingSlot, EmployeeID)
VALUES
(1, 12, 'Anna','Iversen','19:00:00',1),
(2, 12, 'Joakim', 'Iversen', '19:00:00', 1),
(3, 19, 'Vanessa', 'McCarthy', '15:00:00', 3),
(4, 15, 'Marcos', 'Romero', '17:30:00', 4),
(5, 5, 'Hiroki', 'Yamane', '18:30:00', 2),
(6, 8, 'Diana', 'Pinto', '20:00:00', 5);"""

#*******************************************************#
# Insert query to populate "Orders" table:
#*******************************************************#
insert_orders="""
INSERT IGNORE INTO Orders (OrderID, TableNo, MenuID, BookingID, Quantity, BillAmount)
VALUES
(1, 12, 1, 1, 2, 86),
(2, 19, 2, 2, 1, 37),
(3, 15, 2, 3, 1, 37),
(4, 5, 3, 4, 1, 40),
(5, 8, 1, 5, 1, 43);"""

#*******************************************************#
# Insert query to populate "Employees" table:
#*******************************************************#
insert_employees = """
INSERT IGNORE INTO Employees (EmployeeID, Name, Role, Address, Contact_Number, Email, Annual_Salary)
VALUES
(01,'Mario Gollini','Manager','724, Parsley Lane, Old Town, Chicago, IL',351258074,'Mario.g@littlelemon.com','$70,000'),
(02,'Adrian Gollini','Assistant Manager','334, Dill Square, Lincoln Park, Chicago, IL',351474048,'Adrian.g@littlelemon.com','$65,000'),
(03,'Giorgos Dioudis','Head Chef','879 Sage Street, West Loop, Chicago, IL',351970582,'Giorgos.d@littlelemon.com','$50,000'),
(04,'Fatma Kaya','Assistant Chef','132  Bay Lane, Chicago, IL',351963569,'Fatma.k@littlelemon.com','$45,000'),
(05,'Elena Salvai','Head Waiter','989 Thyme Square, EdgeWater, Chicago, IL',351074198,'Elena.s@littlelemon.com','$40,000'),
(06,'John Millar','Receptionist','245 Dill Square, Lincoln Park, Chicago, IL',351584508,'John.m@littlelemon.com','$35,000');"""


# Populate MenuItems table
cursor.execute(insert_menuitems)
connection.commit()
print("Inserting menuitem into table")

# Populate MenuItems table
cursor.execute(insert_menu)
connection.commit()
print("Inserting menu into table")

# Populate Bookings table
cursor.execute(insert_bookings)
connection.commit()
print("Inserting bookings into table")

# Populate Orders table
cursor.execute(insert_orders)
connection.commit()
print("Inserting orders into table")

# Populate Employees table
cursor.execute(insert_employees)
connection.commit()
print("Inserting employees into table")
print("""The database "little_lemon" is ready for use.""")

Inserting menuitem into table
Inserting menu into table
Inserting bookings into table
Inserting orders into table
Inserting employees into table
The database "little_lemon" is ready for use.


In [32]:
peak_hours_procedure = """
CREATE PROCEDURE PeakHours()

BEGIN

SELECT 
HOUR(BookingSlot) AS Hours,
COUNT(HOUR(BookingSlot)) AS no_bookings 
FROM Bookings
GROUP BY Hours
ORDER BY no_bookings DESC;

END
"""

# Execute the query 
cursor.execute(peak_hours_procedure)

# Call the stored procedure with its name
cursor.callproc("PeakHours")

# Retrieve recrods in "dataset"
results = next( cursor.stored_results() ).fetchall()

# Retrieve column names using list comprehension in a 'for' loop 
for column_id in cursor.stored_results():
    columns = [ column[0] for column in column_id.description ]

# Print column names
print(columns)

# Print data 
for result in results:
    print(result)

['Hours', 'no_bookings']
(19, 2)
(15, 1)
(17, 1)
(18, 1)
(20, 1)


In [50]:
# Stored procedure name >> GuestStatus
# Our stored procedure query is
guest_status_procedure ="""
CREATE PROCEDURE GuestStatus()
BEGIN

DECLARE EXIT HANDLER FOR SQLEXCEPTION
BEGIN
    ROLLBACK;
    SELECT 'An error has occurred, operation rollbacked and the stored procedure was terminated';
END;

SELECT 
CONCAT(GuestFirstname,' ',GuestLastname) AS CustomerName,
CASE
WHEN Role IN ("Manager", "Assistant Manager") THEN "Ready to pay"
WHEN Role IN ("Head Chef") THEN "Ready to serve" 
WHEN Role IN ("Assistant Chef") THEN "Preparing Order" 
WHEN Role IN ("Head Waiter") THEN "Order served"
END AS Status
FROM Bookings
LEFT JOIN Employees ON Bookings.EmployeeID=Employees.EmployeeID;
END

"""

# Execute the query
cursor.execute("DROP PROCEDURE IF EXISTS GuestStatus")
cursor.execute(guest_status_procedure)

#********************************************#

# Call the stored procedure with its name
cursor.callproc("GuestStatus")

# Retrieve recrods in "dataset"
results = next( cursor.stored_results() )
dataset = results.fetchall()

# Retrieve column names using list comprehension in a 'for' loop 
for column_id in cursor.stored_results():
    columns = [ column[0] for column in column_id.description ]

# Print column names
print(columns)

# Print data 
for data in dataset:
    print(data)

['CustomerName', 'Status']
('Anna Iversen', 'Ready to pay')
('Joakim Iversen', 'Ready to pay')
('Vanessa McCarthy', 'Ready to serve')
('Marcos Romero', 'Preparing Order')
('Hiroki Yamane', 'Ready to pay')
('Diana Pinto', 'Order served')


In [51]:
# Let's close the cursor and the connection
if connection.is_connected():
    cursor.close()
    print("The cursor is closed.")
    connection.close()
    print("MySQL connection is closed.")
else:
    print("Connection is already closed")

The cursor is closed.
MySQL connection is closed.
